In [1]:
%load_ext autoreload
%autoreload 2

import torch
import pyro
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bb_opt.src.bayesian_opt import (
    optimize,
    get_model_bnn,
    train_model_bnn,
    partial_train_model_bnn,
)
from bb_opt.src.utils import get_path
from gpu_utils.utils import gpu_init

gpu_id = gpu_init()
print(f"Running on GPU {gpu_id}")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

%matplotlib inline

Running on GPU 1


In [2]:
def jointplot(predicted, true, title: str=""):
    if isinstance(predicted, torch.Tensor):
        predicted = predicted.detach().cpu().numpy()

    if isinstance(true, torch.Tensor):
        true = true.detach().cpu().numpy()
        
    ax = sns.jointplot(predicted, true, s=3, alpha=0.5)
    ax.set_axis_labels('Predicted', 'True')
    ax.ax_marg_x.set_title(title)
    return ax

In [3]:
project = "dna_binding"
dataset = "crx_ref_r1"
batch_size = 1
retrain_every = 1000000
partial_steps = 20

root = "/cluster/nhunt/github/bb_opt/"
data_dir = get_path(root, "data", project, dataset)
inputs = np.load(get_path(data_dir, "inputs.npy"))
labels = np.load(get_path(data_dir, "labels.npy"))

top_k_percent = 1

In [4]:
bnn_model, guide = get_model_bnn(n_inputs=inputs.shape[1])
optimizer = pyro.optim.Adam({})

model_name = "../models/crx_1000"
pyro.get_param_store().load(f"{model_name}.params")
optimizer.load(f"{model_name}.opt")

inputs = torch.tensor(inputs, dtype=torch.float32, device=device)
labels = torch.tensor(labels, dtype=torch.float32, device=device)

In [7]:
ps = pyro.get_param_store()
ps.get_all_param_names()

dict_keys(['g_0.weight_location', 'g_0.weight_log_scale', 'g_0.bias_location', 'g_0.bias_log_scale', 'g_2.weight_location', 'g_2.weight_log_scale', 'g_2.bias_location', 'g_2.bias_log_scale'])

In [ ]:
ps[""]

In [1]:
from time import time, sleep
from IPython import get_ipython
from IPython.core.magic import register_line_cell_magic
from IPython.display import Audio, display
from IPython.core.display import HTML

from IPython.core import magic_arguments
from IPython.core.magic import line_cell_magic, Magics, magics_class
 
@magics_class
class NotificationMagics(Magics):
    @line_cell_magic
    @magic_arguments.magic_arguments()
    @magic_arguments.argument('--url', '-u',
          help='URL of sound to play
    )
    def notify(self, line, cell=None, url: str='http://www.soundjay.com/button/beep-07.wav'):
        if cell is None:
            ret = self.shell.ex(line)
        else:
            ret = self.shell.ex(cell)

        audio = Audio(url=url, autoplay=True)
        display(HTML("<style>audio{display:none}</style>"))
        display(audio)

        return ret

In [2]:
ip = get_ipython()
ip.register_magics(NotificationMagics)

In [3]:
%notify print("hi")

hi


In [11]:
%%notify
z = 3

In [12]:
print(x + z)

8
